# Day 3

## Problem 1 

Were recovering valid instructions amidst the corrupted data. The instructions are in the form of a string. Theres a standard way to do this kind of recursive parsing but is overkill for this question so just going simple.

In [1]:
import logging
import re
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

with open('./data/day3_input.txt') as f:
    program = f.read().splitlines()

PATTERN = r"mul\((\d{1,3}),(\d{1,3})\)"
pattern = re.compile(PATTERN)

def process_line(line):
    match = pattern.findall(line)
    return sum([int(a)* int(b) for a,b in match])


running_sum = 0
for line in program:
    running_sum += process_line(line)

running_sum

173517243

## Problem 2

Here we will parse `don't()` and `do()` instructions that enable our inclusion of `mult()` instructions.

Ugh - we can reuse our function above, if we just split a line into segments to search and segments to ignore first. 

But that has us going through the line twice. We can do it in one pass by keeping track of the segments we are in and the segments we are ignoring....

Maybe we writre a custom character parser?! Ugh.... no....

In [2]:
import re
import logging
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger()

# PART 1 SOLUTION
PATTERN = r"mul\((\d{1,3}),(\d{1,3})\)"
pattern = re.compile(PATTERN)

def process_positive_segment(line):
    logger.debug(f"processing line: {line}")
    match = pattern.findall(line)
    return sum([int(a)* int(b) for a,b in match])

# PART 2 SOLUTION

# state: symbol that toggles state
state_map = {True: "do()", False: "don't()"}

# state: function that should processes the segment
state_fn_map = {True: process_positive_segment, 
                False: lambda x: 0 # ignore negative segments
                } 

def process_line(segment, state=True):
    # get the symbol that toggles the current state
    toggler = state_map[not state]
    logger.debug(f"state: {state} toggler: {toggler}")

    # split the segment into two pieces using the toggler
    pieces = segment.split(toggler, 1)
    logger.debug(f"pieces: {pieces}")

    # process the first piece according to the current state
    sum = state_fn_map[state](pieces[0])

    # if there is a second piece, process it with the opposite state since its been toggled
    if len(pieces) > 1:
        sum += process_line(pieces[1], not state)
        
    return sum

### Example

In [3]:
# reset the logger
logger = logging.getLogger()
while logger.hasHandlers():
    logger.removeHandler(logger.handlers[0])
logging.basicConfig(level=logging.DEBUG)

# test the function
line = "xmul(2,4)&mul[3,7]!^don't()_mul(5,5)+mul(32,64](mul(11,8)undo()?mul(8,5))"
line = "dontmul(2,4) do() mul(3,7) do()mul(5,5) mul(32,64) mul(11,8) mul(8,5)"
process_line(line)

DEBUG:root:state: True toggler: don't()
DEBUG:root:pieces: ['dontmul(2,4) do() mul(3,7) do()mul(5,5) mul(32,64) mul(11,8) mul(8,5)']
DEBUG:root:processing line: dontmul(2,4) do() mul(3,7) do()mul(5,5) mul(32,64) mul(11,8) mul(8,5)


2230

### Full problem

In [4]:
# reset the logger
logger = logging.getLogger()
while logger.hasHandlers():
    logger.removeHandler(logger.handlers[0])
logging.basicConfig(level=logging.INFO)


# process the entire program
with open('./data/day3_input.txt') as f:
    program = f.read()

running_sum = 0
for line in [program]:
    running_sum += process_line(line, True)
running_sum

100450138

## Notes

I interpreted this "program" as processing each line independently! For Part 2, I got the wrong answer doing this! I had to make the whole program be a single line and didnt want to rewrite it so I just did a lazy:

```python
program = f.read() # not .readlines()
...
for line in [program]:
```

Also got to clean up the logger switching modes for next time.